In [1]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np

In [2]:
reactions=['love','wow','angry','sad','care']

In [4]:
data=pd.read_csv("../data/Data_cleaning_with_content.csv")
data=data.dropna()
contents=data['contentclean'].values
love=data['love'].values
wow=data['wow'].values
sad=data['sigh'].values
angry=data['grrr'].values
care=data['care'].values
category=data['category'].values
sum_reaction=data['sumreactions'].values

In [5]:
love=love/sum_reaction
wow=wow/sum_reaction
angry=angry/sum_reaction
sad=sad/sum_reaction
care=care/sum_reaction


In [6]:
Emotions_svr={"love":love,"wow":wow,"angry":angry,"sad":sad,"care":care}

In [7]:
Emotions=np.array([love]).reshape(-1,1)
# Emotions

In [8]:

# Emotions=np.insert(Emotions,0,love,axis=1)
Emotions=np.insert(Emotions,1,wow,axis=1)
Emotions=np.insert(Emotions,2,angry,axis=1)
Emotions=np.insert(Emotions,3,sad,axis=1)
Emotions=np.insert(Emotions,4,care,axis=1)
# Emotion_mining=np.array(Emotions)

In [9]:
Emotions.shape

(20443, 5)

# Represent Data Using TFIDF For SVM

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
def tfidf_features(X_train):

    tfidf_vectorizer=TfidfVectorizer(min_df=2, max_df=0.3, ngram_range=(2, 3))
    tfidf_vectorizer.fit(X_train)
    X_train=tfidf_vectorizer.transform(X_train)
#     X_test=tfidf_vectorizer.transform(X_test)
    
    return X_train,tfidf_vectorizer
#, X_test

# Bulid SVR For Emotion Mining

In [12]:
from sklearn.preprocessing import LabelEncoder
def encodin_label(ytrain):
    encoder = LabelEncoder()
    y_train_nn = encoder.fit_transform(ytrain)
    return y_train_nn

In [13]:
# encoder = LabelEncoder()
# encoder.fit(category)
# encoder.classes_

In [14]:
from sklearn import svm
import time
clf_Emotion = svm.SVR()
# clf_Emotion.probability=True
# clf_Sentiment = svm.SVC(kernel='precomputed')
# clf_Sentiment.probability=True
clf_Category = svm.SVC()
clf_Category.probability=True
def train_svc(clf,x_train,y_train):
    X_train,tfidfvectorize=tfidf_features(x_train)
#     Y_train=encodin_label(y_train)
    s=time.time()
    print("-"*10)

    clf.fit(X_train,y_train)
    print("finished",time.time()-s)
    return clf,tfidfvectorize
def predict_svm(clf,post,tfidf):
    post=tfidf_vectorizer.transform([post])
    return clf.predict_proba(post)
    
# clf_Emotion,TF_emotion=train_svc(clf_Emotion,contents,care)
# clf_Sentiment,TF_Sentiment=train_svc(clf_Sentiment,X_train,Y_train)
# clf_Category,TF_Category=train_svc(clf_Category,X_train,Y_train)
def get_all_vectors_for_post_emotion_category_sentiment(post):
    return predict_svm(clf_Emotion,post,TF_emotion),predict_svm(clf_Sentiment,post,TF_Sentiment),predict_svm(clf_Category,post,TF_Category)



import pickle

In [ ]:
def train_svm():
    i=0
    for k,v in Emotions_svr.items():
        
        clf_Emotion_,TF_emotion_=train_svc(clf_Emotion,contents,v)
        with open("../models/SVR_"+str(k)+".pickle","wb") as file:
            pickle.dump(clf_Emotion_,file)
        if i<1:
            i=i+1
            
            with open("../models/TF_Vecotrize.pickle","wb") as file:

                pickle.dump(TF_emotion_,file)
        
    clf_Emotion_,TF_emotion_=train_svc(clf_Category,contents,category)
    with open("../models/SVC_category.pickle","wb") as file:
        pickle.dump(clf_Emotion_,file)

train_svm()


    


----------
finished 315.6224138736725
----------
finished 291.98644256591797
----------
finished 270.83100867271423
----------
finished 330.81699562072754
----------
finished 227.58900356292725
----------


# Build Cnn Lstm Category

In [30]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.models import Sequential
from keras import layers
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
def convert_one_hot(ytrain):
    encoder = LabelEncoder()
    y_train_nn = encoder.fit_transform(ytrain)
    y_train_nn
    encoders = OneHotEncoder(sparse=False)
    y_train_nn = y_train_nn.reshape(len(y_train_nn), 1)
    y_train_nn=encoders.fit_transform(y_train_nn)
    return y_train_nn
maxlen=100
tokenizer = Tokenizer(num_words=15000)

tokenizer.fit_on_texts(contents)


X_train = tokenizer.texts_to_sequences(contents)
    # X_test = tokenizer.texts_to_sequences(sentences_test)

        # Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

        # Pad sequences with zeros
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
Y_train=convert_one_hot(category)
def train_nn(model,Y_train):
    maxlen=100
    tokenizer = Tokenizer(num_words=15000)

    tokenizer.fit_on_texts(contents)


    X_train = tokenizer.texts_to_sequences(contents)
    # X_test = tokenizer.texts_to_sequences(sentences_test)

        # Adding 1 because of reserved 0 index
    vocab_size = len(tokenizer.word_index) + 1

        # Pad sequences with zeros
    X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)

    model.fit(X_train,Y_train,batch_size=128,epochs=10)
    return model
def creat_model_cnn(vocab_size):
    
    model = Sequential()
    model.add(layers.Embedding(vocab_size, 50, input_length=100))
    model.add(layers.Conv1D(128, 5, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(13,activation="softmax"))
    model.compile(optimizer='adam',
                        loss='categorical_crossentropy',
                         metrics=['accuracy'])
    return model
def creat_model_lstm(vocab_size):
    
    model = Sequential()
    model.add(layers.Embedding(vocab_size, 50, input_length=100))
    model.add(layers.Bidirectional(layers.LSTM(256)))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(13,activation="softmax"))
    model.compile(optimizer='adam',
                        loss='categorical_crossentropy',
                         metrics=['accuracy'])
    return model
model_category_cnn=creat_model_cnn(vocab_size)
model_category_cnn=train_nn(model_category_cnn,Y_train)
model_category_cnn.save("../models/CNN_category.h5")
model_category_lstm=creat_model_lstm(vocab_size)
model_category_lstm=train_nn(model_category_cnn,Y_train)
model_category_lstm.save("../models/LSTM_category.h5")

In [25]:
len(reactions)

5

In [26]:
def creat_model_cnn_E(vocab_size):
    
    model = Sequential()
    model.add(layers.Embedding(vocab_size, 50, input_length=100))
    model.add(layers.Conv1D(128, 5, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(len(reactions)))
    model.compile(optimizer='adam',
                        loss='mse',
                         metrics=['accuracy'])
    return model
def creat_model_lstm_E(vocab_size):
    
    model = Sequential()
    model.add(layers.Embedding(vocab_size, 50, input_length=100))
    model.add(layers.Bidirectional(layers.LSTM(256)))
#     model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(len(reactions)))
    model.compile(optimizer='adam',
                        loss='mse',
                         metrics=['accuracy'])
    return model
model_category_cnn=creat_model_cnn_E(vocab_size)
model_category_cnn=train_nn(model_category_cnn,Emotions)
model_category_cnn.save("../models/CNN_Emotion.h5")
model_category_lstm=creat_model_lstm_E(vocab_size)
model_category_lstm=train_nn(model_category_cnn,Emotions)
model_category_lstm.save("../models/LSTM_Emotion.h5")

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
160/160 [==============================] - 20s 119ms/step - loss: 0.0916 - accuracy: 0.5659
Epoch 2/10
160/160 [==============================] - 20s 125ms/step - loss: 0.0741 - accuracy: 0.5831
Epoch 3/10
160/160 [==============================] - 19s 117ms/step - loss: 0.0610 - accuracy: 0.6598
Epoch 4/10
160/160 [==============================] - 19s 116ms/step 

In [32]:
with open("../models/tokenizer_NN.pickle","wb") as file:
    pickle.dump(tokenizer,file)


# CNN Emotion

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.models import Sequential
from keras import layers

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
def convert_one_hot(ytrain):
#     encoder = LabelEncoder()
    y_train_nn = encoder.transform(ytrain)
    y_train_nn
    encoders = OneHotEncoder(sparse=False)
    y_train_nn = y_train_nn.reshape(len(y_train_nn), 1)
    y_train_nn=encoders.fit_transform(y_train_nn)
    return y_train_nn
Y_train=convert_one_hot(category)

In [ ]:
Emotions.shape

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
maxlen=100
tokenizer = Tokenizer(num_words=15000)

tokenizer.fit_on_texts(contents)


X_train = tokenizer.texts_to_sequences(contents)
# X_test = tokenizer.texts_to_sequences(sentences_test)

    # Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

    # Pad sequences with zeros
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)

In [ ]:
model = Sequential()
model.add(layers.Embedding(vocab_size, 50, input_length=100))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(13,activation="softmax"))
model.compile(optimizer='adam',
                loss='mse',
                 metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(X_train,Y_train,batch_size=128,epochs=10)

In [169]:
model.save("CNN_category.h5")

In [152]:
with open("tokenizer_NN.pickle","wb") as file:
    pickle.dump(tokenizer,file)
    

In [172]:
model.predict(X_train[0:1])

array([[1.3491851e-03, 8.1063056e-04, 1.1725579e-03, 3.2122436e-04,
        6.5769744e-04, 2.0318530e-03, 3.0358115e-04, 4.8772068e-05,
        1.6519595e-03, 5.3208960e-06, 9.9078685e-01, 8.4135431e-04,
        1.8980825e-05]], dtype=float32)

In [50]:
X_train.shape

(20057, 100)